In [89]:
import pandas as pd
from sklearn.metrics import accuracy_score

In [90]:
results = pd.read_csv("all_predictions.csv")

In [91]:
results.columns

Index(['Unnamed: 0', 'Text', 'Label', 'bert_cased_finetuned_-1',
       'bert_cased_finetuned_0', 'bert_cased_finetuned_1',
       'bert_cased_finetuned_pred',
       'bert_pretrain_output_all_notes_150000_finetuned_-1',
       'bert_pretrain_output_all_notes_150000_finetuned_0',
       'bert_pretrain_output_all_notes_150000_finetuned_1',
       'bert_pretrain_output_all_notes_150000_finetuned_pred',
       'bert_pretrain_output_disch_100000_finetuned_-1',
       'bert_pretrain_output_disch_100000_finetuned_0',
       'bert_pretrain_output_disch_100000_finetuned_1',
       'bert_pretrain_output_disch_100000_finetuned_pred',
       'bert_uncased_finetuned_-1', 'bert_uncased_finetuned_0',
       'bert_uncased_finetuned_1', 'bert_uncased_finetuned_pred',
       'biobert_1.0_finetuned_-1', 'biobert_1.0_finetuned_0',
       'biobert_1.0_finetuned_1', 'biobert_1.0_finetuned_pred',
       'biobert_1.1_finetuned_-1', 'biobert_1.1_finetuned_0',
       'biobert_1.1_finetuned_1', 'biobert_1.1_fin

In [92]:
test_data = pd.read_csv("drugsComTest_raw.tsv", sep="\t")
test_data.columns

Index(['Unnamed: 0', 'drugName', 'condition', 'review', 'rating', 'date',
       'usefulCount'],
      dtype='object')

In [93]:
cb_all_results = results[["Text", "Label", 'bert_pretrain_output_disch_100000_finetuned_-1', 'bert_pretrain_output_disch_100000_finetuned_1', "bert_pretrain_output_disch_100000_finetuned_pred"]]
cb_all_results.columns = ["Text", "Label", "C_Disch_-1", "C_Disch_1", "C_Disch_Pred"]

In [94]:
cb_all_results = pd.concat([cb_all_results, test_data[["drugName","condition","date","usefulCount","rating"]]], axis=1)

In [95]:
cb_all_results.head()

,Text,Label,C_Disch_-1,C_Disch_1,C_Disch_Pred,drugName,condition,date,usefulCount,rating
0,I&#039;ve tried a few antidepressants over the...,1,0.000106,0.996484,1,Mirtazapine,Depression,"February 28, 2012",22,10.0
1,My son has Crohn&#039;s disease and has done v...,1,0.000061,0.999480,1,Mesalamine,"Crohn's Disease, Maintenance","May 17, 2009",17,8.0
2,Quick reduction of symptoms,1,0.000057,0.999158,1,Bactrim,Urinary Tract Infection,"September 29, 2017",3,9.0
3,Contrave combines drugs that were used for alc...,1,0.000066,0.999632,1,Contrave,Weight Loss,"March 5, 2017",35,9.0
4,I have been on this birth control for one cycl...,1,0.000056,0.999118,1,Cyclafem 1 / 35,Birth Control,"October 22, 2015",4,9.0


In [96]:
accuracy_score(cb_all_results["Label"], cb_all_results["C_Disch_Pred"])

0.8887958933154781

In [104]:
cb_all_results.shape[0]

53766

In [98]:
misclassified = cb_all_results[cb_all_results["Label"]!=cb_all_results["C_Disch_Pred"]]
misclassified.shape[0]

5979

In [107]:
1-(727/5979)

0.8784077604950661

In [108]:
5979-727

5252

In [99]:
misclassified_as_pos = cb_all_results[(cb_all_results["Label"]==-1)&(cb_all_results["C_Disch_Pred"]==1)]
misclassified_as_pos.shape[0]

390

In [100]:
misclassified_as_pos_sample = misclassified_as_pos.sample(100, axis=0, random_state=0)
misclassified_as_pos_sample.to_csv("sentiment_misclassified_as_pos_sample.csv")

In [102]:
misclassified_as_neg = cb_all_results[(cb_all_results["Label"]==1)&(cb_all_results["C_Disch_Pred"]==-1)]
misclassified_as_neg.shape[0]

337

In [103]:
misclassified_as_neg_sample = misclassified_as_neg.sample(100, axis=0, random_state=0)
misclassified_as_neg_sample.to_csv("sentiment_misclassified_as_neg_sample.csv")

In [88]:
#for x in misclassified_as_pos.iterrows():
#    print(x[0])
#    print("drugname:", x[1]["drugName"])
#    print("condition:", x[1]["condition"])
#    print("useful count:", x[1]["usefulCount"])
#    print("positive prob:", x[1]["CB_All_1"])
#    print(x[1]["Text"])
#    print("")

In [123]:
misclassified_as_pos_sample_analyzed = pd.read_csv("sentiment_misclassified_as_pos_sample 50analyzed.csv")
misclassified_as_pos_sample_analyzed["error_analysis"].value_counts()/misclassified_as_pos_sample_analyzed["error_analysis"].value_counts().sum()

another drug mentioned                0.40
mixed sentiment                       0.34
wrong prediction                      0.14
mislabeled by reviewer                0.06
stopping drug reduced side effects    0.04
no pertinent review content           0.02
Name: error_analysis, dtype: float64

In [124]:
misclassified_as_neg_sample_analyzed = pd.read_csv("sentiment_misclassified_as_neg_sample 50analyzed.csv")
misclassified_as_neg_sample_analyzed["error_analysis"].value_counts()/misclassified_as_neg_sample_analyzed["error_analysis"].value_counts().sum()

negative side effects          0.62
another drug mentioned         0.22
original symptoms mentioned    0.12
cost                           0.02
could be improved              0.02
Name: error_analysis, dtype: float64

In [126]:
337/5979

0.05636394045827062